In [4]:
import re
from time import sleep
import itertools
import requests
from pathlib import Path
from bs4 import BeautifulSoup

URLS = {
    "hist_data": "https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/daily/kl/historical/",
    "recent_data": "https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/daily/kl/recent/",
}

# for category, url in URLS.items():
#     page = requests.get(url)
#     soup = BeautifulSoup(page.content, "html.parser")

#     links = soup.find_all("a", href=re.compile(".pdf$|.txt$|.zip$"))

#     for link in links:
#         path = Path("./data/") / category
#         path.mkdir(parents=True, exist_ok=True)
#         file_path = path / link["href"]
#         mode = "w+b" if "pdf" or "zip" in link["href"] else "w+"
#         file_url = url + link["href"]
#         if not file_path.is_file():
#             with requests.get(file_url) as r:
#                 with open(str(file_path), mode) as f:
#                     f.write(r.content)
#                     sleep(0.5)


In [17]:
# from zipfile import ZipFile

# for category in URLS:
#     path = Path("./data/") / category
#     path.mkdir(parents=True, exist_ok=True)

#     for file in path.glob("*.zip"):
#         with ZipFile(file, "r") as zip:
#             fname = [fname for fname in zip.namelist() if "produkt_klima_tag" in fname]
#             zip.extract(fname[0], Path(path / "extracts"))


In [34]:
import pandas as pd
import pyarrow as pa
from pathlib import Path

path = Path("./data/") / "recent_data" / "extracts"

dtypes= {
    'STATIONS_ID': "object", 
    'MESS_DATUM': "object", 
    'QN_3': "category", 
    'QN_4': "category", 
    ' RSK': "int32", 
    'RSKF': "category", 
    'SHK_TAG': "int8",
    '  NM': "category", 
    ' TMK': "float32", 
    ' UPM': "float32", 
    ' TXK': "float32", 
    ' TNK': "float32", 
    ' TGK': "float32",
}

df_lst = []
for file in path.glob("*.txt"):
    df_lst += [pd.read_table(
        file, 
        sep=";",
        usecols=['STATIONS_ID', 'MESS_DATUM', 'QN_3', 'QN_4', ' RSK',
        'RSKF', 'SHK_TAG', '  NM', ' TMK', ' UPM',
        ' TXK', ' TNK', ' TGK'],
        engine="pyarrow",
        dtype=dtypes,
    )]


# print(df.info())

In [37]:
df = pd.concat(df_lst)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 308461 entries, 0 to 549
Data columns (total 13 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   STATIONS_ID  308461 non-null  object 
 1   MESS_DATUM   308461 non-null  object 
 2   QN_3         308461 non-null  int64  
 3   QN_4         308461 non-null  int64  
 4    RSK         308461 non-null  int32  
 5   RSKF         308461 non-null  int64  
 6   SHK_TAG      308461 non-null  int8   
 7     NM         308461 non-null  float64
 8    TMK         308461 non-null  float32
 9    UPM         308461 non-null  float32
 10   TXK         308461 non-null  float32
 11   TNK         308461 non-null  float32
 12   TGK         308461 non-null  float32
dtypes: float32(5), float64(1), int32(1), int64(3), int8(1), object(2)
memory usage: 23.8+ MB


In [47]:
df = df.replace(-999, pa.null())
df.MESS_DATUM = pd.to_datetime(df.MESS_DATUM, format="%Y%m%d")
df2 = df.astype(dtypes)
df2.info()

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'pyarrow.lib.DataType'